<a href="https://colab.research.google.com/github/InvoLab101/10conv-poc/blob/main/10conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load data

##### Install dependencies

In [ ]:
!pip install openai
!pip install diskcache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 747.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 798.6 kB/s eta 0:00:00


##### load data

In [9]:
from google.colab import drive
import sys
import os
import pandas as pd
import json
import openai
import logging
import diskcache

drive.mount('/content/drive')
conversation_folder = '/content/drive/MyDrive/10conversations'  # Change this to your desired directory
conversation_files = os.listdir(conversation_folder)
QUESTIONS = [
    'Did the HDA welcome the customer to the bank, introduced himself/herself by name, and attained customer\'s name immediately? (Yes or No)',
    'Did the HDA acknowledge the customer’s request, paraphrased back to demonstrate active listening and understanding of the customer’s request or needs? (Yes or No)',
    'Did the HDA demonstrate empathy of customer\'s concern (Yes or No)',
    'Did the HDA make strong/confident customer advocacy statements? (Yes or No)',
    'Did the HDA promote passcode for future interactions? (Yes or No or NA)',
    'Did the HDA gain permissions to ask questions and make value added recommendations? (Yes or No)',
    'Did the HDA solve the customer’s request efficiently and position alternatives with customer benefits and educate customer as needed? (Yes or No)',
    'Was a call transfer absolutely necessary to resolve the customer’s request? (Yes or No or NA)',
    'Did the HDA add value by promoting or offering or acknowledging digital banking use and services? (Yes or No)',
    'Did the HDA use benefit statements to explain why the product or service is of value to the customer’s needs and ask for the business? (Yes or No)',
    'Did the HDA accurately recap the conversation, provide a clear explanation of next steps setting expectations, and check for additional needs? (Yes or No)',
    'Did the HDA thank the customer by name for their loyalty to the bank and close with appreciation for their business? (Yes or No)'
    ]

conversations = []
for filename in conversation_files:
    with open(os.path.join(conversation_folder, filename), "r") as file:
        conversation = file.read()
        conversations.append(conversation)

data = pd.DataFrame({"Conversation": conversations})

print(f"loaded {len(data['Conversation'])} conversations")
print(f"loaded {len(QUESTIONS)} questions")

with open("/content/drive/MyDrive/work_asu/config.json", "r") as config_file:
    config = json.load(config_file)
openai.api_key = config["openai_api_key"]
print('openAi key loaded')

cache = diskcache.Cache("/content/drive/MyDrive/work_asu")
print("cache loaded")

def promtAskAllQuestionForOneConversation(conversation):
    prompt = '''
    below is a customer service call between a Help Desk Agent(HDA) and Customer(C)

    {}
    based on the conversation above, answer these questions:

    {}
    Only give me the answers in order and put them in a list

    '''.format(conversation, '\n'.join(QUESTIONS))
    logging.debug("engineered prompt: {}".format(prompt))
    return prompt


def get_openai_answer(conversation, modelName):
    response = openai.Completion.create(
      # model="text-davinci-003",
      model = modelName,
      prompt=promtAskAllQuestionForOneConversation(conversation),
      temperature=0,
      max_tokens=50,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
    )
    logging.info(response)
    return response.choices[0].text

def get_openai_answer_with_disk_cache(conversation, modelName):
    cache_key = (conversation, modelName)
    answers = ''
    # Check if the answer is in the cache
    if cache_key in cache:
        print('in cache')
        answers = cache[cache_key]
    else:
      print('not in cache, need to call openai')
      # If not in cache, fetch the answer and store it in the cache
      answers = get_openai_answer(conversation, modelName)  # Implement your OpenAI API call here
      answers = answers.strip('\n').strip('.').split(',')
      if (len(answers)!= len(QUESTIONS)):
            raise Exception("anwsers count invalid")
    cache[cache_key] = answers
    print(answers)
    return answers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
loaded 10 conversations
loaded 12 questions
openAi key loaded
cache loaded


##### answer questions

In [10]:
cache.clear()
data["OpenAI_Answers"] = data["Conversation"].apply(lambda conv: get_openai_answer_with_disk_cache(conv, 'text-davinci-003'))
print(data)

not in cache, need to call openai
['Yes', ' Yes', ' Yes', ' No', ' NA', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No']
not in cache, need to call openai
['Yes', ' Yes', ' Yes', ' No', ' NA', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No']
not in cache, need to call openai
['Yes', ' Yes', ' Yes', ' Yes', ' NA', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No']
not in cache, need to call openai
['Yes', ' Yes', ' Yes', ' No', ' NA', ' Yes', ' Yes', ' NA', ' No', ' No', ' Yes', ' No']
not in cache, need to call openai
['Yes', ' Yes', ' Yes', ' Yes', ' NA', ' Yes', ' Yes', ' NA', ' Yes', ' Yes', ' Yes', ' Yes']
not in cache, need to call openai
['Yes', ' Yes', ' Yes', ' No', ' NA', ' Yes', ' No', ' No', ' No', ' No', ' Yes', ' No']
not in cache, need to call openai
['Yes', ' Yes', ' Yes', ' No', ' NA', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No']
not in cache, need to call openai
['Yes', ' Yes', ' Yes', ' No', ' NA', ' Yes', ' Yes', ' No', ' No', ' No', ' Yes', ' No']
not 

### Helper functions